### Get the ttbar root file

In [1]:
! curl -s -k -L -O https://uaf-3.t2.ucsd.edu/~atuna/muoncollider/data/mlpf/ttbar/ttbar_reco_0_edm4hep.root
! ls -lthr

total 87M
-rw-rw-r-- 1 atuna atuna  24 Jan 29 17:29 README.md
-rw-rw-r-- 1 atuna atuna  16 Jan 29 18:36 requirements.txt
drwxrwxr-x 7 atuna atuna   7 Jan 29 18:37 env
-rw-rw-r-- 1 atuna atuna 18K Jan 29 21:55 muon_collider_notebook.ipynb
-rw-rw-r-- 1 atuna atuna 87M Jan 30 12:41 ttbar_reco_0_edm4hep.root


### Print collections in the root file

In [2]:
import uproot

In [3]:
with uproot.open("ttbar_reco_0_edm4hep.root") as fi:
    print(fi.keys())

    tree = fi["events"]
    for key in tree.keys():
        if "/" in key or key.startswith("_"):
            continue
        array = tree[key].array()
        print(f"{key}: {array}")

['events;1', 'metadata;1', 'podio_metadata;1']
AllCaloHitContributionsCombined: [{'AllCaloHitContributionsCombined.PDG': [11, ..., 2212], ...}]
AllTracks: [{'AllTracks.type': [0, 0, 0, 0, ..., 0, 0, 0], 'AllTracks.chi2': [...], ...}]
AllTracks_dQdx: [{'AllTracks_dQdx.dQdx.type': [0, 0, 0, ..., 0, 0], ...}]
ECalBarrelCollection: [{'ECalBarrelCollection.cellID': [...], ...}]
EcalBarrelCollectionConed_objIdx: [{'EcalBarrelCollectionConed_objIdx.index': [0, ...], ...}]
EcalBarrelCollectionDigi: [{'EcalBarrelCollectionDigi.cellID': [148, ...], ...}]
EcalBarrelCollectionRec: [{'EcalBarrelCollectionRec.cellID': [148, ..., 7341204], ...}]
EcalBarrelCollectionSel_objIdx: [{'EcalBarrelCollectionSel_objIdx.index': [51, ...], ...}]
EcalBarrelRelationsSimConed: [{'EcalBarrelRelationsSimConed.weight': [1, 1, 1, 1, 1, ..., 1, 1, 1, 1]}]
EcalBarrelRelationsSimDigi: [{'EcalBarrelRelationsSimDigi.weight': [1, 1, 1, 1, 1, ..., 1, 1, 1, 1]}]
EcalBarrelRelationsSimRec: [{'EcalBarrelRelationsSimRec.weight':

### Define constants for tracking (expert detail)

In [4]:
# https://github.com/iLCSoft/LCIO/blob/master/src/cpp/include/pre-generated/EVENT/TrackState.h#L35-L38
AtIP = 1
AtFirstHit = 2
AtLastHit = 3
AtCalorimeter = 4
location_choice = AtIP

### Print info about tracks, clusters, and MC particles

In [5]:
do_tracks = True
do_clusters = True
do_mcparticles = True

with uproot.open("ttbar_reco_0_edm4hep.root") as fi:
    i_event = 0

    print("*"*50)
    print(f"Event {i_event}")

    if do_tracks:
        ndf = fi["events"]["SiTracks_Refitted/SiTracks_Refitted.ndf"].array()[i_event]
        begin = fi["events"]["SiTracks_Refitted/SiTracks_Refitted.trackStates_begin"].array()[i_event]
        end = fi["events"]["SiTracks_Refitted/SiTracks_Refitted.trackStates_end"].array()[i_event]
        location = fi["events"]["_SiTracks_Refitted_trackStates/_SiTracks_Refitted_trackStates.location"].array()[i_event]
        mask = (location == location_choice)
        phi = fi["events"]["_SiTracks_Refitted_trackStates/_SiTracks_Refitted_trackStates.phi"].array()[i_event][mask]
        omega = fi["events"]["_SiTracks_Refitted_trackStates/_SiTracks_Refitted_trackStates.omega"].array()[i_event][mask]
        tanLambda = fi["events"]["_SiTracks_Refitted_trackStates/_SiTracks_Refitted_trackStates.tanLambda"].array()[i_event][mask]

        print("*"*50)
        print("SiTracks_Refitted n =", len(ndf))
        print("SiTracks_Refitted ndf", ndf)
        print("SiTracks_Refitted begin", begin)
        print("SiTracks_Refitted end", end)
        print("SiTracks_Refitted location", location)
        print("SiTracks_Refitted phi", phi)
        print("SiTracks_Refitted omega", omega)
        print("SiTracks_Refitted tanLambda", tanLambda)

    if do_clusters:
        energy = fi["events"]["PandoraClusters/PandoraClusters.energy"].array()[i_event]
        itheta = fi["events"]["PandoraClusters/PandoraClusters.iTheta"].array()[i_event]
        phi = fi["events"]["PandoraClusters/PandoraClusters.phi"].array()[i_event]
        position_x = fi["events"]["PandoraClusters/PandoraClusters.position.x"].array()[i_event]
        position_y = fi["events"]["PandoraClusters/PandoraClusters.position.y"].array()[i_event]
        position_z = fi["events"]["PandoraClusters/PandoraClusters.position.z"].array()[i_event]

        print("*"*50)
        print("PandoraClusters n =", len(energy))
        print("PandoraClusters energy", energy)
        print("PandoraClusters position_x", position_x)
        print("PandoraClusters position_y", position_y)
        print("PandoraClusters position_z", position_z)


    if do_mcparticles:
        pdg = fi["events"]["MCParticle/MCParticle.PDG"].array()[i_event]
        generatorStatus = fi["events"]["MCParticle/MCParticle.generatorStatus"].array()[i_event]
        charge = fi["events"]["MCParticle/MCParticle.charge"].array()[i_event]
        mass = fi["events"]["MCParticle/MCParticle.mass"].array()[i_event]
        px = fi["events"]["MCParticle/MCParticle.momentum.x"].array()[i_event]
        py = fi["events"]["MCParticle/MCParticle.momentum.y"].array()[i_event]
        pz = fi["events"]["MCParticle/MCParticle.momentum.z"].array()[i_event]

        print("*"*50)
        print("MCParticle n =", len(pdg))
        print("MCParticle pdg", pdg)
        print("MCParticle generatorStatus", generatorStatus)
        print("MCParticle charge", charge)
        print("MCParticle mass", mass)
        print("MCParticle px", px)
        print("MCParticle py", py)
        print("MCParticle pz", pz)



**************************************************
Event 0
**************************************************
SiTracks_Refitted n = 161
SiTracks_Refitted ndf [19, 1, 19, 13, 15, 17, 7, 19, 17, 17, ..., 1, 17, 7, 5, 17, 21, 3, 19, 11, 11]
SiTracks_Refitted begin [0, 4, 8, 12, 16, 20, 24, 28, 32, ..., 612, 616, 620, 624, 628, 632, 636, 640]
SiTracks_Refitted end [4, 8, 12, 16, 20, 24, 28, 32, 36, ..., 616, 620, 624, 628, 632, 636, 640, 644]
SiTracks_Refitted location [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, ..., 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]
SiTracks_Refitted phi [-1.43, -1.59, -1.57, -1.54, -1.63, -1.58, ..., 1.41, 1.23, 1.28, 1.27, 1.68]
SiTracks_Refitted omega [-0.000873, 0.00034, 0.000218, -3.96e-05, ..., 0.000142, 0.000555, 0.002]
SiTracks_Refitted tanLambda [-0.506, -0.367, -0.332, -0.328, -0.326, ..., 0.445, 0.457, 0.448, 0.478, 0.502]
**************************************************
PandoraClusters n = 314
PandoraClusters energy [0.0505, 3.79, 0.262, 18.8, 0.0552, 0.116, .